<a href="https://colab.research.google.com/github/Meyyaou/pythondaily/blob/main/STEP4-Webscrape/webscrape_project_(minione).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#import all tools
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml
import urllib.parse

In [20]:
#from search page to individually each page
url="https://www.amazon.com/s?k=manga&rh=n%3A4367&ref=nb_sb_noss"
header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7,ar;q=0.6'}

In [22]:
page=requests.get(url,headers=header)
soup=BeautifulSoup(page.text, 'lxml')
page

<Response [503]>

In [ ]:
#to get readable html post parsing
soup.prettify()

'<!DOCTYPE html>\n<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us">\n <!-- sp:feature:head-start -->\n <head>\n  <script>\n   var aPageStart = (new Date()).getTime();\n  </script>\n  <meta charset="utf-8"/>\n  <!-- sp:end-feature:head-start -->\n  <!-- sp:feature:csm:head-open-part1 -->\n  <script type="text/javascript">\n   var ue_t0=ue_t0||+new Date();\n  </script>\n  <!-- sp:end-feature:csm:head-open-part1 -->\n  <!-- sp:feature:cs-optimization -->\n  <meta content="on" http-equiv="x-dns-prefetch-control"/>\n  <link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>\n  <link href="https://m.media-amazon.com" rel="dns-prefetch"/>\n  <link href="https://completion.amazon.com" rel="dns-prefetch"/>\n  <!-- sp:end-feature:cs-optimization -->\n  <!-- sp:feature:csm:head-open-part2 -->\n  <script type="text/javascript">\n   window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(fu

In [ ]:
def get_book_info(url):
    page=requests.get(url,headers=header)
    soup=BeautifulSoup(page.text, 'lxml')
    #get title
    title_elem= soup.select_one("#productTitle")
    title=title_elem.text if title_elem else None
    #print(title)
    #get author
    author_elem=soup.select_one('span.author')
    author=author_elem.text if author_elem else None
    #print(author)
    #get date de sortie
    date_elem=soup.select_one("#rpi-attribute-book_details-publication_date > div.a-section.a-spacing-none.a-text-center.rpi-attribute-value > span")

    date=date_elem.text.strip() if date_elem else None
    #print(date)
    #get price prime deal
    full_price_elem= soup.select_one('span.a-price-whole')
    frac_price_elem=soup.select_one('span.a-price-fraction')
    curr_elem=soup.select_one("span.a-price-symbol")
    price=full_price_elem.text+frac_price_elem.text+curr_elem.text if full_price_elem and frac_price_elem and curr_elem else None
    #print(price)

    #get ordre in top vente
    order_bs_elem= soup.select_one("div.zg-badge-wrapper > a > i")
    order_bs=order_bs_elem.text if order_bs_elem else None

    cat_bs_elem=soup.select_one("div.zg-badge-wrapper > a > span > span")
    cat_bs=cat_bs_elem.text if cat_bs_elem else None
    #print(order_bs, cat_bs)

    #get price livraison ?wherfe is itttt
    #get price kindle?
    return {
        'title': title,
        'author': author,
        'date': date,
        'price': price,
        'order_bs': order_bs,
        'cat_bs': cat_bs
    }

In [ ]:
counter = 0

def search_res_listing(url_search, books_data):
    global counter
    response = requests.get(url_search, headers=header)
    search_soup = BeautifulSoup(response.text, 'lxml')
    url_elem_list = search_soup.select('[data-asin] h2 a')

    for url_elem in url_elem_list:
        if counter >= 20:
            return books_data
        url = urllib.parse.urljoin(url_search, url_elem.attrs.get("href"))
        book_info = get_book_info(url)
        books_data.append(book_info)
        counter += 1

    next_book_el = search_soup.select_one('a:-soup-contains("Next")')
    if next_book_el and counter < 20:
        next_book_url = next_book_el.attrs.get('href')
        next_book_url = urllib.parse.urljoin(url_search, next_book_url)
        books_data = search_res_listing(next_book_url, books_data)

    return books_data

In [ ]:
s_url="https://www.amazon.com/s?k=manga&rh=n%3A4367&ref=nb_sb_noss"
books_data = search_res_listing(s_url, [])
dataframe = pd.DataFrame(books_data)
dataframe.to_csv("mangas_amazon_test1.csv", index=False, sep=',')

In [ ]:
#tronquer what we dont need! => was when we searched for infos from results page, not consequent!
#for header in soup.find_all('header'):
#  header.decompose()
#soup.find('div', class_='navLeftFooter').decompose()
#soup.find_all('div', class_='navLeftFooter')
#soup.find('div', class_='rhf-frame').decompose()
#soup.find('div', class_='copilot-secure-display').decompose()

In [ ]:
#get the title
#titles=[]
#pattern =re.compile(r'\d+')

#for title in soup.find_all('span', class_='a-text-normal'):
#  if not pattern.match(title.string):
#    titles.append(title.string)
#    title
#titles

In [ ]:
#get auteur
#auteurs=[]

#for auteur in soup.find_all('span', class_='a-size-base'):
#  auteurs.append(auteur.string)
#auteurs


In [ ]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [ ]:
dataframe["author"]

KeyError: 'author'

In [ ]:
df=dataframe

In [ ]:
#DATA preprocessing: format des champs
#date
df['date']= pd.to_datetime(df['date'])
#
auths= df['author']
for i,auth in enumerate(auths):
  auth_name=""
  auth=auth.split(" ")
  auth_name+=auth[0]
  auth_name+=' '
  auth_name+=auth[1] if 'Author' not in auth[1] else ""
  auth_name=auth_name.split("\n")[1]
  df.loc[i,"author"]=auth_name
#print(df['author'])

In [ ]:
import numpy as np

In [ ]:
df.head()
#cat_bs drop the in keep list of cats and if none=> No bs
df['cat_bs']=df['cat_bs'].fillna("No bs")
cats=df['cat_bs']
cat_bs = []
for cat in cats:
  if cat != 'No bs':
    words=str(cat).split(' ')
    filtr_words=[]
    i=0
    if words[0]=='in':
      i=1
    while i <len(words):
      if words[i]=="&":
        if i>0 and filtr_words:
          filtr_words[-1]=filtr_words[-1]+' '+words[i+1]
        i+=2
      else:
        filtr_words.append(words[i])
        i+=1
    cat_bs.append(filtr_words)
  else:
    cat_bs.append(["No bs"])
#print(cat_bs)
df['cat_bs']=cat_bs

In [ ]:
df.head()

In [ ]:
#order_bs if none=> No bs
df['order_bs']=df['order_bs'].fillna("No bs")

In [ ]:
#DATA ANALYSIS some basic plots
import matplotlib.pyplot as plt
import seaborn as sns

#annee de publication
plt.hist(df['date'],bins=20)

#df['date'].hist()

In [ ]:
#price
plt.figure()
plt.plot(df['price'], c='green', marker='o', linestyle='-')
plt.xlabel('Index')
plt.ylabel('Price ($)')
plt.tight_layout()
plt.show()

In [ ]:
#between order_bs and cat_bs
crosstab = pd.crosstab(df['order_bs'], df['cat_bs'])

plt.figure(figsize=(8,6))
sns.heatmap(crosstab, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
#email alert stuff
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [ ]:
def email_alert(subject,body,to):
  msg=MIMEMultipart()
  msg.attach(MIMEText(body, 'plain'))
  msg['subject']=subject
  msg['to']=to
  user='x'
  msg['from']=user

  pwd='x'

  server=smtplib.SMTP("smtp.gmail.com",587)
  server.starttls()
  server.login(user,pwd)
  server.send_message(msg)
  server.quit()

In [ ]:
#email_alert('hey', "hello world it's a test man", 'rec@gmail.com')

In [ ]:
#variation analysis
def variation_analysis(old_data_path, new_data_path):
    old_data = pd.read_csv(old_data_path)
    new_data = pd.read_csv(new_data_path)

    for ind, row in new_data.iterrows():
        old_row = old_data[old_data['title'] == row['title']]

        if not old_row.empty:
            old_row = old_row.iloc[0]

            if row['price'] != old_row['price']:
                email_alert('Price Changing', f"the price of {row['title']} has changed from {old_row['price']} to {row['price']}", 'rec@gmail.com')

            if row['order_bs'] != old_row['order_bs']:
                email_alert('Order of Best Seller Changing', f"{row['title']} that had {old_row['order_bs']} as an order of best selling, in {old_row['cat_bs']} category(ies) before, has now an order of {row['order_bs']} in {row['cat_bs']} category(ies)", 'amyya929@gmail.com')
        else:
            email_alert('New manga added', f"A new manga has appeared in your best sellers search results: {row['title']} by {row['author']} published in {row['date']} is at position {row['order_bs']} in category {row['cat_bs']}.", 'amyya929@gmail.com')

    for ind, row in old_data.iterrows():
        if row['title'] not in new_data['title'].values:
            email_alert('Manga disappeard !', f"the manga {row['title']} by {row['author']}, published in {row['date']} is no longer a best-seller", 'amyya929@gmail.com')

    new_data.to_csv('manga_amazon_test3.csv', index=False)

In [ ]:
old_data=pd.read_csv('mangas_amazon_test1.csv')
new_data=pd.read_csv('mangas_amazon_test2.csv')
variation_analysis(old_data, new_data)